# Paulo Geolocate

In [ ]:
import pandas as pd
import geopandas as gpd

# Import border line file
borderLine = gpd.read_file('CaaporaFragmentEdge.gpkg', layer='EdgeLine')
border = borderLine.unary_union

# Import data, should be lat long decimal ex: -7.35, -34.51
df = pd.read_excel('Paula POINT loc.xlsx')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf = gdf.set_crs('EPSG:4326')
gdf = gdf.to_crs('EPSG:31985')

# Calculate distance of each point in the group to the centroid and border
# Calcular a distância de cada ponto no grupo para o centroide e fronteira
for row in gdf['geometry']:
    # Apply to the scan geodataframe / Aplicar ao geodataframe de varredura
    # gdf.loc[:,'distCentr'] = gdf.distance(centroid[0])
    gdf.loc[:,'distBorder'] = gdf.distance(border)

# cenList = gdf['distCentr'].tolist()
borList = gdf['distBorder'].tolist()
gdf.to_file('POINT loc.gpkg', driver='GPKG', layer='points')
gdf.to_csv('POINT loc.csv', index=False)
gdf.head()

# Natsumi Geolocate & Border Dist

### Coordinate Fixer

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Import raw data as excel file
df = pd.read_excel('natsumiData.xlsx')
df = df.drop(index=0)
df.reset_index(inplace=True, drop=True)

# south initial coordinate conversion
df['dms'] = df['Coordenadas S'].str.strip()

# split deg min sec into variables
deg = df['dms'].str[:2]
min = df['dms'].str[3:5]
sec = df['dms'].str[6:10]

df['deg'] = deg
df['min'] = min
df['sec'] = sec

df['deg'] = df['deg'].astype(float)
df['min'] = df['min'].astype(float)
df['sec'] = df['sec'].astype(float)

# convert/merge the standalone values to deimal degrees and apply -1 for south
df['Coordenadas S'] = (df['deg'] + (df['min']/60) + (df['sec']/3600)) * -1

# west initial coordinate conversion
df['dms'] = df['Coordenadas W'].str.strip()

deg = df['dms'].str[:3]
min = df['dms'].str[4:6]
sec = df['dms'].str[7:11]

df['deg'] = deg
df['min'] = min
df['sec'] = sec

df['deg'] = df['deg'].astype(float)
df['min'] = df['min'].astype(float)
df['sec'] = df['sec'].astype(float)

# convert/merge the standalone values to deimal degrees and apply -1 for south
df['Coordenadas W'] = (df['deg'] + (df['min']/60) + (df['sec']/3600)) * -1

# south final coordinate conversion
df['dms'] = df['Coordenadas S.1'].str.strip()

deg = df['dms'].str[:2]
min = df['dms'].str[3:5]
sec = df['dms'].str[6:10]

df['deg'] = deg
df['min'] = min
df['sec'] = sec

df['deg'] = df['deg'].astype(float)
df['min'] = df['min'].astype(float)
df['sec'] = df['sec'].astype(float)

# convert/merge the standalone values to deimal degrees and apply -1 for south
df['Coordenadas S.1'] = (df['deg'] + (df['min']/60) + (df['sec']/3600)) * -1

# west final coordinate conversion
df['dms'] = df['Coordenadas W.1'].str.strip()

deg = df['dms'].str[:3]
min = df['dms'].str[4:6]
sec = df['dms'].str[7:11]

df['deg'] = deg
df['min'] = min
df['sec'] = sec

df['deg'] = df['deg'].astype(float)
df['min'] = df['min'].astype(float)
df['sec'] = df['sec'].astype(float)

# convert/merge the standalone values to deimal degrees and apply -1 for south
df['Coordenadas W.1'] = (df['deg'] + (df['min']/60) + (df['sec']/3600)) * -1

# remov the temp columns for deg min sec
df.pop('dms')
df.pop('deg')
df.pop('min')
df.pop('sec')

# to gdf with geometry as initial
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Coordenadas W'],df['Coordenadas S'])) 
# set crs/projection
gdf = gdf.set_crs('EPSG:4326')
gdf = gdf.to_crs('EPSG:31985')
gdf['Coordenadas Inicial'] = gdf['geometry']
# create points for final
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(df['Coordenadas W.1'],df['Coordenadas S.1'])) 
gdf = gdf.set_crs('EPSG:4326', allow_override=True)
gdf = gdf.to_crs('EPSG:31985')
gdf['Coordenadas Final'] = gdf['geometry']
gdf.pop('geometry')

# import edge lines
edgeLine = gpd.read_file('CaaporaFragmentEdge.gpkg', layer='EdgeLine')
edgeupperLine = gpd.read_file('CaaporaFragmentEdge.gpkg', layer='EdgeUpper')
edgelowerLine = gpd.read_file('CaaporaFragmentEdge.gpkg', layer='EdgeLower')

edge = edgeLine.unary_union
edgeupper = edgeupperLine.unary_union
edgelower = edgelowerLine.unary_union

gdf.to_csv('preBorder.csv', index=False)

gdf.head()

,Hora final,Nome,Duração (s),Categoria,Observação,STATUS,Coordenadas S,Coordenadas W,Coordenadas S.1,Coordenadas W.1,Identificação,Coordenadas Inicial,Coordenadas Final
0,11:04:00,021921 11.04 SAD rap,56.0,SUB,NaN,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086)
1,11:11:00,021921 11.11 J3,289.0,J3,NaN,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086)
2,11:13:00,021921 11.13 FF,146.0,FF,Nome no BORIS FF 021921 11.13 2m26s,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086)
3,11:21:00,021921 11.21 J2 rap,159.0,J2,NaN,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086)
4,11:26:00,021921 11.26 J1,295.0,J1,j1 pequeno; últimos scan no inferior,Transcrito,-7.524889,-34.963000,-7.524889,-34.963000,NaN,POINT (283394.742 9167739.763),POINT (283394.742 9167739.763)


In [ ]:
# fig, ax = matplotlib.pyplot.subplots(figsize=(14, 10))
# gdf_da.plot(column = 'Low_Inc_%', ax = ax)

In [7]:
# Dist for initial geometry
gdf.loc[:,'iniDistEdge'] = gdf['Coordenadas Inicial'].distance(edge)
gdf.loc[:,'iniDistUpEdge'] = gdf['Coordenadas Inicial'].distance(edgeupper)
gdf.loc[:,'iniDistLowEdge'] = gdf['Coordenadas Inicial'].distance(edgelower)

# Dist for final geometry
gdf.loc[:,'finDistEdge'] = gdf['Coordenadas Final'].distance(edge)
gdf.loc[:,'finDistUpEdge'] = gdf['Coordenadas Final'].distance(edgeupper)
gdf.loc[:,'finDistLowEdge'] = gdf['Coordenadas Final'].distance(edgelower)

gdf.to_csv('natsumiData.csv')

gdf.head()

,Hora final,Nome,Duração (s),Categoria,Observação,STATUS,Coordenadas S,Coordenadas W,Coordenadas S.1,Coordenadas W.1,Identificação,Coordenadas Inicial,Coordenadas Final,iniDistEdge,iniDistUpEdge,iniDistLowEdge,finDistEdge,finDistUpEdge,finDistLowEdge
0,11:04:00,021921 11.04 SAD rap,56.0,SUB,NaN,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086),64.943795,64.943795,156.098147,64.943795,64.943795,156.098147
1,11:11:00,021921 11.11 J3,289.0,J3,NaN,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086),64.943795,64.943795,156.098147,64.943795,64.943795,156.098147
2,11:13:00,021921 11.13 FF,146.0,FF,Nome no BORIS FF 021921 11.13 2m26s,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086),64.943795,64.943795,156.098147,64.943795,64.943795,156.098147
3,11:21:00,021921 11.21 J2 rap,159.0,J2,NaN,Transcrito,-7.525111,-34.963194,-7.525111,-34.963194,NaN,POINT (283373.389 9167715.086),POINT (283373.389 9167715.086),64.943795,64.943795,156.098147,64.943795,64.943795,156.098147
4,11:26:00,021921 11.26 J1,295.0,J1,j1 pequeno; últimos scan no inferior,Transcrito,-7.524889,-34.963000,-7.524889,-34.963000,NaN,POINT (283394.742 9167739.763),POINT (283394.742 9167739.763),38.502222,38.502222,160.803798,38.502222,38.502222,160.803798
